In [ ]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

list_nft = {}

for i in range(0, 100):
    nftId = f"NFT{i}"
    list_nft[nftId] = f"../nft_images/{nftId}.png"

# Affichage des 5 premières images avec leur ID
fig, axes = plt.subplots(1, 5, figsize=(15, 5))

for idx, (nftId, imagePath) in enumerate(list_nft.items()):
    if idx >= 5:
        break
    try:
        image = Image.open(imagePath)
        axes[idx].imshow(image)
        axes[idx].set_title(nftId)
        axes[idx].axis('off')
    except FileNotFoundError:
        print(f"Image non trouvée : {imagePath}")
        axes[idx].set_title(f"{nftId}\n[Manquante]")
        axes[idx].axis('off')

plt.show()

In [ ]:
# Import des modules nécessaires
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake
from smart_contract import SmartContractDefinition, SmartContractWritingOperation

# Création de plusieurs wallets pour les nœuds de la blockchain
walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

# Création d'un wallet propriétaire pour le smart contract
walletProprietaire = Wallet()

#Création de wallets user :
walletMatthieu = Wallet()
walletEdouard = Wallet()
walletMilo = Wallet()
walletHugo = Wallet()
walletVincent = Wallet()
walletHenri = Wallet()

# Initialisation du mécanisme de Proof of Stake avec le premier nœud
pos = ProofOfStake(walletNode1.publicKey)

#On reset le network
Node.reset_network()

#On connecte les 3 noeuds au network
node1 = BlockchainNode(walletNode1,pos)
node2 = BlockchainNode(walletNode2,pos)
node3 = BlockchainNode(walletNode3,pos)

assert len(Node.get_all_nodes())==3
"Success"

'Success'

In [ ]:
# Exemple de source de code pour le smart contract
source_code = """
class MyNFTContract:
    def __init__(self, issuerPublicKey):
        self.issuerPublicKey = issuerPublicKey
        self.nfts = {}  # Dictionnaire des NFTs

    def mint(self, ownerPublicKey, nftId):
        if nftId not in self.nfts:
            self.nfts[nftId] = ownerPublicKey
            print(f"NFT {nftId} frappé par {ownerPublicKey}")
        else:
            raise ValueError(f"NFT {nftId} déjà existant")

    def transfer(self, senderPublicKey, receiverPublicKey, nftId):
        if nftId in self.nfts and self.nfts[nftId] == senderPublicKey:
            self.nfts[nftId] = receiverPublicKey
            print(f"NFT {nftId} transféré de {senderPublicKey} à {receiverPublicKey}")
        else:
            raise ValueError("NFT inexistant ou propriété invalide")
"""

# Création du Smart Contract (définition)
NFT_contract = SmartContractDefinition(issuerPublicKey=walletProprietaire.publicKey, sourceCode=source_code, Ntoken=5)

# Signer le certificat
walletProprietaire.sign(NFT_contract)

# Instanciation du Smart Contract
NFT_instance = NFT_contract.instantiate_contract()

# Ajouter le Smart Contract au premier nœud (node1)
node1.new_certificate(NFT_contract) # Ajouter le smart contract au nœud

print(node1.blockchain.blockList)
print(node2.blockchain.blockList)
print(node3.blockchain.blockList)

Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x0000027244D6F140>


In [ ]:
#On vérifie si la période de minage est ouverte
NFT_contract.is_minting_open()

False

La période de minage est fermée donn les utilisateurs ne peuvent pas encore miner leur NFT. On peut quand même faire les tests

In [ ]:
NFT_contract.mint(ownerPublicKey=walletEdouard.publicKey, nftId="NFT1")

ValueError: La période de minage est fermée.

Il n'y a que le propriétaire du smart contract qui peut activer la période de minage. Ici c'est Alice !

In [ ]:
NFT_contract.open_minting(walletProprietaire.publicKey,1000)

Période de minage ouverte pendant 1000 secondes


In [ ]:
NFT_contract.is_minting_open()

True

Maintenant qu'on a ouvert la periode de minage les utilisateurs peuvent miner des NFT

In [ ]:
NFT_contract.mint(ownerPublicKey=walletEdouard.publicKey, nftId="NFT1")
print(NFT_contract.owners)

NFT NFT1 frappée par 30820122300d06092a864886f70d01010105000382010f003082010a0282010100da4b264bba1456417e86fb0cabc1f23fdaef4df7ffc1dca355419528318814a69f589ec63fa2a0b302343058d8fe3d802005791b564907f5faff4d308352b302c629e9283396a35b5869a02a6bf938a48ce5ebf720fd820662759c1ef4980642d919990edca849bc9c87b2b3b4b7276c585ab2d9f25b21fc4c15d7c96bac0f97cb6a3e8bc3457ca40bbecc7e118324d5454a38c48ea2ba4a21093bed2dcf583d6a0e516c95485a699117e808f7a58ccae4806337f860f759683884478e226da34bf68cd229bc04d50345575ca6b4efda24fdbb380f602a1d93662b38fc2c459b6adef7041d2d483d51df9f03b11a99143f1e2a6b1e7ef901e939f40ff3cb2bf30203010001
{'NFT1': '30820122300d06092a864886f70d01010105000382010f003082010a0282010100da4b264bba1456417e86fb0cabc1f23fdaef4df7ffc1dca355419528318814a69f589ec63fa2a0b302343058d8fe3d802005791b564907f5faff4d308352b302c629e9283396a35b5869a02a6bf938a48ce5ebf720fd820662759c1ef4980642d919990edca849bc9c87b2b3b4b7276c585ab2d9f25b21fc4c15d7c96bac0f97cb6a3e8bc3457ca40bbecc7e118324d5454a38c48ea2ba4a21093bed2d

In [ ]:
NFT_contract.mint(ownerPublicKey=walletMatthieu.publicKey, nftId="NFT2")
NFT_contract.mint(ownerPublicKey=walletHugo.publicKey, nftId="NFT3")
NFT_contract.mint(ownerPublicKey=walletMilo.publicKey, nftId="NFT4")
NFT_contract.mint(ownerPublicKey=walletHenri.publicKey, nftId="NFT5")

NFT NFT2 frappée par 30820122300d06092a864886f70d01010105000382010f003082010a02820101008fbe6463901abbaff1a379d2a01759fdd1cdad3fd6b0aeeed754e79e160bbcbe69a838fed3fc5ccb51222f0b720d2a5ad5ab045432d98fcb43729ffad34dcded482d773043dd1c31b3a8f1916fabf6d3c7a49782848ad2c866cbb3665cb16a226790d16e91cfc9ede254848fdd893d1148017c7dfdfcbe8b95457d9446c3fdb3b2e8337a809f17bd0a9f90da0d83722df1e7b8e6285ec0a8e0751da2376c106d90b9afdbfec76001b12c6ff5cd482dd90dfc3d80beb6a255ab452910f5a4b46b6756b3395a8c95d0f773822f12fd6d0be40920b72e060a5819d3532a3bc75a7a0aaaeed880a55720407c518cbf975db94a67e95f9f68e1d8160a93ce972f7b9f0203010001
NFT NFT3 frappée par 30820122300d06092a864886f70d01010105000382010f003082010a0282010100c710b9a96d9a4447ba04ae12ce31f7b8570bfc39c16e4861494dcc4f13152816396c25bed928bbd1a22aa3cd9ba90c05fc78dc95cfe05211ac2ec675b741a0f86bf4b7a7f328e775bc3f496468c924ce4f85aa971535e5add95619798d2d41b30bd8004ec25f85773e3e91be5c1844859f61d1f8f14a67ce1ab169f9729e4acaa61cce5aa1b5eb21a7d420c75ddba3054a006d24315e311

In [ ]:
NFT_contract.mint(ownerPublicKey=walletVincent.publicKey, nftId="NFT6")

ValueError: Nombre de token maximal atteint

Ici on a atteind le nombre maximal de token donc on ne peut plus acheter de NFT

On peut changer le propriétaire d'un NFT si c'est le propriétaire de ce dernier qui le demande

In [ ]:
NFT_contract.transfer(senderPublicKey=walletHenri.publicKey, receiverPublicKey=walletVincent.publicKey, nftId="NFT5")

NFT NFT5 transférée de 30820122300d06092a864886f70d01010105000382010f003082010a0282010100cc9ec47411fc31907c6ce0436e67709abc86eadaef3d5ca3af18c49ab413c6b8f3ca9725004bdde21060742e0dd750e239e4aeaf73e363818ad51a9d0d6259c2f05884932725035f04e4704fe5c754465596fdcf7156be330b90f63b371da66e1c501331298893e8dbba7e911e12d28c213a8b87e8c5872b079ce64669038b65917502a38ff8d9a3c67b60904837856b739e5a559dff44cca51c7a2ab8862f5ff0c82e1a68f2ce825f89f0959c179b5df741cc2827edc470a684ea59e8a6d9315467bc0178691672326deeee820e773c88ef1a0c7fecacff5b009c934a27473aba580378f039ff16dee4a89b7debfd9e5a108e659b542f3c767a02225a41d0630203010001 à 30820122300d06092a864886f70d01010105000382010f003082010a0282010100b999e22954225b2da49aa89ede0469a52b1bcb45738f00db5fb03c2fe6beb313b0ec881984ddaf6ca554e559e64c5c3bbc0135292f92124af2829b778d62958a0a16f3619fadcbe9c8747576b64cedab6d65d18c1d7faa294b1fc5b6d32fffed6ae83bf4602c35fc6e73936852cd357034b9e0fbefe1abea4d4661d70aafc62dcc1ec347699d6e11bafa09c0d98064e8b53400749fb4b0cc5e5b79749dbecd8e

In [ ]:
NFT_contract.transfer(senderPublicKey=walletHenri.publicKey, receiverPublicKey=walletVincent.publicKey, nftId="NFT5")

ValueError: NFT n'appartient pas à l'expéditeur